전처리 및 특성엔지니어링

In [ ]:
# 각 파일마다 Segment 변수를 추가해주는 코드


customer_train_df = pd.read_parquet('/content/drive/MyDrive/Dacon/customer_train_cleaned.parquet')
customer_train_df_seg = customer_train_df[['기준년월','ID','Segment']]

credit_with_seg = customer_train_df_seg.merge(credit_train_df, on=['기준년월', 'ID'], how='left') # segment 변수 추가
#credit_with_seg.to_parquet('/content/drive/MyDrive/DACON/rawdata/credit_with_seg.parquet')

In [ ]:
# 동일 데이터 삭제


# 데이터 다 동일한 변수 4개 삭제!
columns_drop = ['상품관련면제카드수_B0M','임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']
customer_train_df = customer_train_df.drop(columns=columns_drop)

In [ ]:
# 결측치 ( 삭제 , 기타(없음))


# 결측치 많은 변수 삭제!
customer_train_df = customer_train_df.drop(columns=['최종카드발급일자','최종유효년월_신용_이용가능', '최종유효년월_신용_이용'])

##### 1. 결측치 처리
# '_1순위신용체크구분', '_2순위신용체크구분’
#    → 결측치 '기타'로 채우기
#    → '_1순위신용체크구분_인코딩', '_2순위신용체크구분_인코딩’
customer_train_df['_1순위신용체크구분'] = customer_train_df['_1순위신용체크구분'].fillna('기타')
customer_train_df['_2순위신용체크구분'] = customer_train_df['_2순위신용체크구분'].fillna('기타')
mapping = {'신용': 1, '체크': 0, '기타': -1}
customer_train_df['1순위신용체크구분_인코딩'] = customer_train_df['_1순위신용체크구분'].map(mapping)
customer_train_df['2순위신용체크구분_인코딩'] = customer_train_df['_2순위신용체크구분'].map(mapping)
customer_train_df.drop(columns=['_1순위신용체크구분','_2순위신용체크구분'], inplace=True)

In [ ]:
# 이진화


# 직장시도명 → 직장_수도권여부
customer_train_df['직장_수도권여부'] = customer_train_df['직장시도명'].isin(['서울', '경기']).astype(int)
customer_train_df = customer_train_df.drop(columns=['직장시도명'])

# '카드론이용여부' 변수 생성
sales_with_seg['카드론이용여부'] = (
    (sales_with_seg['최초카드론이용경과월'] != 999) &
    (sales_with_seg['최종카드론이용경과월'] != 999)
).astype(int)

In [ ]:
# A들의 한도금액 수준 최소치를 만들어 기준치를 삼음


# '카드이용한도금액_A수준복합' 변수 추가
def classify_dual_limit(row):
    b1 = row['카드이용한도금액_B1M'] >= 170344
    b2 = row['카드이용한도금액_B2M'] >= 170344
    if b1 and b2:
        return 2     # 둘 다 170,344 이상
    elif b1 or b2:
        return 1     # 하나만 170,344 이상
    else:
        return 0    # 둘 다 170,344 미만

In [ ]:
# 구간별 점수 부여


# '이용금액대_점수' 변수 생성 (상관계수 -0.6026)
amount_mapping = {
    '01.100만원+': 18,
    '02.50만원+': 5,
    '03.30만원+': 2,
    '04.10만원+': 2,
    '05.10만원-': 1,
    '09.미사용': 1
}

sales_with_seg['이용금액대_점수'] = sales_with_seg['이용금액대'].map(amount_mapping)

# 원래 변수 '이용금액대' 삭제
sales_with_seg = sales_with_seg.drop(columns=['이용금액대'])

모델링

In [ ]:
# XGBoost 기준 중요도 상위 300개 변수 추출
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': temp_model.feature_importances_
}).sort_values(by='importance', ascending=False)

top300_features = importance_df.head(300)['feature'].tolist()

print(top300_features)

top300_df = pd.DataFrame({'feature': top300_features})
top300_df.to_csv(
    "/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv",
    index=False,
    encoding="utf-8-sig"
)

In [ ]:
# 300개 변수로 SMOTE 사용해 오버샘플링하고 , 가중치(weight) 부여 코드예시로 XGBoost만가져옴


train_df = pd.read_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()
y = y.map({'A':0, 'B':1,'C':2,'D':3,'E':4})
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

# 변수 300개 사용
top300_df = pd.read_csv("/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv")
top300_features = top300_df['feature'].tolist()
X_top300 = X[top300_features]

# 오버샘플링
smote = SMOTE(sampling_strategy={0: 30000, 1: 30000, 2: 250000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_top300, y)

# 클래스별 weight 계산
classes = np.unique(y_resampled)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_resampled)
class_weights = dict(zip(classes, weights))
sample_weights = pd.Series(y_resampled).map(class_weights)

for cls in sorted(class_weights):
    print(f"클래스 {cls}: weight = {class_weights[cls]:.2f}")

In [ ]:
# 3개모델 soft voting


# test 데이터 준비
X_test = test_df[top300_features]

# XGB predict_proba (iteration_range)
proba_xgb = xgb_model_loaded.predict_proba(X_test, iteration_range=(0, 5000))

# LGB Booster (predict가 proba)
proba_lgb = lgb_model_loaded.predict(X_test, num_iteration=3348)

# CAT predict_proba
proba_cat = cat_model_loaded.predict_proba(X_test)

# soft voting
ensemble_proba = (0.4 * proba_xgb) + (0.3 * proba_lgb) + (0.3 * proba_cat)
ensemble_preds = np.argmax(ensemble_proba, axis=1)
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
ensemble_preds_label = pd.Series(ensemble_preds).map(inverse_label_map)